In [ ]:
import os
from improc.io import parse_collection, DCAccessor
DCAccessor.register()

import numpy as np
import holoviews as hv
hv.extension('bokeh')

# read example images

In [ ]:
from skimage.io import imread
import os

basedir = '../../data'

img_files = ['2D/201031JD002regimetestAAB_20201031_173042/TIF_OVR_MIP/201102JD002AAB_201031_174004_B02_T0001F001L01A03Z01C01.tif',
             '2D/201031JD002regimetestAAB_20201031_173042/TIF_OVR_MIP/201102JD002AAB_201031_174004_B02_T0001F001L01A02Z01C02.tif',
             '2D/201031JD002regimetestAAB_20201031_173042/TIF_OVR_MIP/201102JD002AAB_201031_174004_B02_T0001F001L01A01Z01C03.tif']

stack_files = ['3D/membrane/P21_Ch0-registered-T0289.tif',
               '3D/nuclei/P21_Ch1-registered-T0289.tif',
               '3D/cell_seg/P21_Ch0-registered-T0289.tif',
               '3D/nuclei_seg/P21_Ch1-registered-T0289.tif',]

imgs = [imread(os.path.join(basedir, f), img_num=0) for f in img_files]
stacks = [imread(os.path.join(basedir, f)) for f in stack_files]

# numpy array to hv dataset

Creates a holoviews dynamic map returning a hv.Dataset and keeps a handle on the input image for live updates

In [ ]:
from inter_view.utils import HvDataset

ds = HvDataset(img=stacks[0], spacing=(2,0.26,0.26), label='membrane')

ds.dmap(), ds.img.shape

# view a single channel

Displays an image along with colormap, intensity and opacity widgets. The image is dynamically rasterized to the displayed resolution depending on the zoom level, i.e. on-the-fly image pyramid (still requires to load the full image in memory on the server side). 

Viewers called with return_panel=True directly return a layout with the image and widgets. When eturn_panel=False, only the modified holoview object is returned. Useful for chaining operations, i.e. slicing volume, applying single channel settings, merging channels, etc.

In [ ]:
from inter_view.view_images import ChannelViewer

hv_dataset = HvDataset(img=imgs[0])
channel_viewer = ChannelViewer(return_panel=True)


channel_viewer(hv_dataset.dmap())

In [ ]:
# Dynamically change the image without rebuilding the entire plot
hv_dataset.img = imgs[2]

# view channels as overlay

In [ ]:
from inter_view.view_images import OverlayViewer

overlay_viewer = OverlayViewer(return_panel=True,
                               channel_viewers={'marker1':ChannelViewer(),
                                                'marker2':ChannelViewer(),
                                                'marker3':ChannelViewer()})

dmaps = [HvDataset(img=img).dmap() for img in imgs]
overlay_viewer(dmaps)

# alternatively, channel viewers can be automatically built from a config dictionary (see below)

# view channels as composite rgb
Composite viewer works the same as overlay viewer except greyscal channels are blended into a rgb image at the end. Currently max blending is used.

In [ ]:
from inter_view.view_images import CompositeViewer

# use channel config definition instead of manually instantiating ChannelViewer
channel_config = {1:{'label':'marker1','cmap':'cyan','intensity_bounds':(200,30000), 'slider_limits':(0,60000)},
                  2:{'label':'marker2','cmap':'magenta','intensity_bounds':(350,20000), 'slider_limits':(0,40000)},
                  3:{'label':'marker3','cmap':'yellow','intensity_bounds':(800,60000), 'slider_limits':(0,60000)}}


composite_viewer = CompositeViewer.from_channel_config(channel_config, return_panel=True)

dmaps = [HvDataset(img=img).dmap() for img in imgs]
composite_viewer(dmaps)

# view segmentation

In [ ]:
from inter_view.view_images import SegmentationViewer
from holoviews import opts

# use channel config definition instead of manually instantiating ChannelViewer
channel_config = {'membrane':{'cmap':'red', 'intensity_bounds':(100,60000), 'slider_limits':(0,60000)},
                  'nuclei':{'cmap':'gray', 'intensity_bounds':(1400,20000), 'slider_limits':(0,60000)},
                  'membrane_seg':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'bitdepth':16, 'opacity':0.2},
                  'nuclei_seg':{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'bitdepth':16, 'opacity':0.2}}

opts.defaults(opts.Image('channel.membrane_seg', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,256*256-1), tools=['hover']),
              opts.Image('channel.nuclei_seg', clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,256*256-1), tools=['hover']))

segmentation_viewer = SegmentationViewer.from_channel_config(channel_config,
                                                             composite_channels=['membrane', 'nuclei'],
                                                             overlay_channels=['membrane_seg', 'nuclei_seg'],
                                                             return_panel=False)

zslice = stacks[0].shape[0]//2
middle_stack_imgs = [s[zslice] for s in stacks]
dmaps = [HvDataset(img=img, label=l).dmap() for img, l in zip(middle_stack_imgs,channel_config.keys())]
dmaps = segmentation_viewer(dmaps)
segmentation_viewer.panel(dmaps)

# view slice
SliceViewer provide a slider to slice a dataset along the specified axis. The sliced dataset cannot be rendred directly and needs to be converted first (e.g. to a hv.Image). This can be accomplished by chaining operations of a ChannelViewer instance.

In [ ]:
from inter_view.view_images import SliceViewer
from holoviews import opts
import panel as pn

# define some default plot config
opts.defaults(opts.Image(cmap='gray', frame_width=600))

hv_dataset = HvDataset(img=stacks[1], spacing=(2,0.26,0.26))

slice_viewer = SliceViewer(axis='y')
channel_viewer = ChannelViewer()

dmap = slice_viewer(hv_dataset.dmap())
dmap = channel_viewer(dmap)

# create a custom layout to show widgets from SliceViewer and ChannelViewer together
pn.Row(dmap, pn.Column(slice_viewer.widgets(), channel_viewer.widgets()))

# View orthogonal slices
Slices the input dataset along x,y,z and returns 3 DynamicMap. The panel() function builds a display panel with synchronized plots + crosshair. the stack can be navigated by clicking on the images

In [ ]:
from inter_view.view_images import OrthoViewer

channel_config = {'membrane':{'cmap':'red', 'intensity_bounds':(100,60000), 'slider_limits':(0,60000)},
                  'nuclei':{'cmap':'gray', 'intensity_bounds':(1400,20000), 'slider_limits':(0,60000)}}

hv_datasets = [HvDataset(img=stack, spacing=(2,0.26,0.26)) for stack in stacks[:2]]

ortho_viewer = OrthoViewer()
composite_viewer = CompositeViewer.from_channel_config(channel_config)

dmaps = [ortho_viewer(ds.dmap()) for ds in hv_datasets]

# invert slices and channels
dmaps = tuple(zip(*dmaps))

dmaps = [composite_viewer(dmap) for dmap in dmaps]
panel = ortho_viewer.panel(dmaps)

# add the composite viewer above the orthoview widget (navigation checkbox)
panel[1][1] = pn.Column(composite_viewer.widgets(), panel[1][1])

panel